# Load Depenency

In [197]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [198]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [199]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [200]:
# disease=input()
# print(disease)

In [201]:
# disease = 'parkinson'

#disease = 'depression'

# disease = 'diabetes'

disease = 'tdcs'



hit mesh some day

Pull ONE

#Explore Data

In [202]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [203]:
type(infodf)

dict

In [204]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

1396


In [205]:
count = ct.get_study_count(search_expr=disease)


In [206]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


In [207]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"]

'NStudiesFound': 3656,

In [208]:
# new_df
new_df.shape

(1396, 20)

In [209]:
new_df.head

<bound method NDFrame.head of     Index        NCTId                                          Condition  \
0       1  NCT03288363                                  Alzheimer Disease   
1       2  NCT02161302                         Chronic Pain|Endometriosis   
2       3  NCT04759092                                  Alzheimer Disease   
3       4  NCT03879044                          Shift-Work Sleep Disorder   
4       5  NCT02665988                 Chronic Pain|Psychiatric Disorders   
..    ...          ...                                                ...   
392  1392  NCT02921061  Mixed Phenotype Acute Leukemia|Previously Trea...   
393  1393  NCT04545294  Schizophrenia|Schizoaffective Disorder|Negativ...   
394  1394  NCT04038788  Schizophrenia|Schizoaffective Disorder|Psychot...   
395  1395  NCT00801489  Acute Myeloid Leukemia With Inv(16)(p13.1q22);...   
396  1396  NCT00416598  Acute Myeloid Leukemia|Acute Myeloid Leukemia ...   

                                         Offi

In [210]:
# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

In [211]:
# print(temp.shape)

In [212]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


In [213]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [214]:
new_df2.shape

(1396, 21)

In [215]:
new_df2

,index,NCTId,IsFDARegulatedDrug,IsFDARegulatedDevice,IsUnapprovedDevice,PrimaryOutcomeMeasure,PrimaryOutcomeDescription,PrimaryOutcomeTimeFrame,SecondaryOutcomeMeasure,SecondaryOutcomeDescription,...,OtherOutcomeMeasure,OtherOutcomeDescription,OtherOutcomeTimeFrame,EligibilityCriteria,StudyPopulation,HealthyVolunteers,ReferencePMID,LocationCity,LocationState,LocationFacility
0,1,NCT03288363,No,No,,Comparing between the active tDCS group with t...,difference of ADAS-Cog psychometric scale scor...,3 months,,,...,,,,Inclusion Criteria:||A man or woman with proba...,,No,,Limoges,,Centre Hospitalier Esquirol
1,2,NCT02161302,,,,Change in worst daily pain assessed with the v...,Patients will be asked to daily write down the...,"Once Daily. Start 7 days before treatment, eve...",Change in serum biomarkers level: The brain de...,A blood sample will be colected to measure BDN...,...,Depressive symptoms level|Anxiety level|Psychi...,Depression symptoms will be assessed by using ...,Treatment Day 1 (before receiving the 1st sess...,"Inclusion Criteria:||age > 18 years old, non-m...",,No,21170277|20945016|18312584|14677403|20633386|2...,Porto Alegre,Rio Grande do Sul,Hospital de Clínicas de Porto Alegre
2,3,NCT04759092,No,No,,To investigate adverse effects of the treatmen...,Investigation of tolerability of long term hom...,4 months|8 months,,,...,,,,Inclusion Criteria:||Age 60-85.|Fulfil diagnos...,,No,,Tromsø,Troms,University Hospital of North Norway
3,4,NCT03879044,No,Yes,,Psychomotor Vigilance Test|Karolinska Sleepine...,Objective measure of sleepiness. Scale ranges ...,3 minutes|5 minutes,Epworth Sleepiness Scale,Subjective measure of sleepiness. The scale ra...,...,,,,Inclusion Criteria:||age 18-65 years|works 5 o...,,No,,Columbus,Ohio,The Ohio State University
4,5,NCT02665988,,,,Change From Baseline in Visual Analogue Scale ...,Visual analogue scale measure for pain. Score ...,On a weekly basis: 1) Day 1 (first day of tDCS...,,,...,,,,Inclusion Criteria:||Males and females greater...,,No,,Houston,Texas,The Pain Management Program at The Menninger C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,1392,NCT02921061,Yes,,,Number of Participants Experiencing Dose Limit...,Evaluated according to National Cancer Institu...,Up to 49 days,Number of Participants With Minimal Residual D...,"Compared to historical controls of filgrastim,...",...,,,,Inclusion Criteria:||For patients with newly d...,,No,32077361,Seattle,Washington,Fred Hutch/University of Washington Cancer Con...
393,1393,NCT04545294,No,No,,The change over time in the negative symptoms ...,A clinician-administered rating scale to measu...,Five weeks,The change over time in the Chinese version of...,A clinician-administered rating scale to measu...,...,,,,Inclusion Criteria:||Eligible participants age...,,No,27023427|25477826|27930934|23993864|25528757|2...,Taipei,,Tri-service general hospital
394,1394,NCT04038788,No,No,,Changes from baseline negative dimension score...,A clinician-administered rating scale to measu...,One month,Changes from baseline scores at Personal and S...,A clinician-administered rating scale to measu...,...,,,,Inclusion Criteria:||Eligible participants age...,,No,33096356,Taipei,,Tri-service general hospital
395,1395,NCT00801489,Yes,No,,Complete remission rate|Toxicity rate,,Up to 2 years|Up to 2 years,,,...,,,,Inclusion Criteria:||Patients must have untrea...,,No,,Houston,Texas,M D Anderson Cancer Center


In [216]:




# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId"],
#     max_studies=10,
#     min_rnk=5,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])
# temp

In [217]:




# # Get the NCTId, Condition and Brief title fields from 500 studies related to Coronavirus and Covid, in csv format.
# api_pull_1 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
#     "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" 
#     ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","IsFDARegulatedDrug","IsFDARegulatedDevice","BriefSummary"],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# df1=pd.DataFrame.from_records(api_pull_1[1:], columns=api_pull_1[0])

# # 
# # df1 = pd.DataFrame.to_frame().reset_index()



In [218]:
# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
# ,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"

# ],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records

# ct.get_study_count(search_expr="Parkinson")

# # Read the csv data in Pandas
# import pandas as pd

# df2 = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

##cleaning

In [219]:
# print(type(df1))

In [220]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='outer')

In [221]:
result.shape

(1396, 40)

In [222]:
# result = result.loc[:,~result.columns.duplicated()]

In [223]:
# result.shape

In [224]:
# result.columns.tolist()

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [225]:
result = result.drop(result.columns[[20]], axis=1)

In [226]:
result.shape

(1396, 39)

In [227]:
# result.columns.tolist()

Save file

In [228]:
# result.to_csv(r'data_raw.csv', index = False)
result.to_excel(r'data_raw.xlsx', index = False)

In [229]:
result.to_hdf('data_raw.h5', key='df', mode='w')  

In [230]:
# labels, levels = pd.factorize(result["Phase"])  

In [231]:
# labels

In [232]:
# fun_barplot()